In [13]:
import os
import pandas as pd
from functions import get_all_origins, mask_qnt, convert_to_gr, substituir_por_comprimidos
import re
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
df = get_all_origins()

pattern = r'(\d+[.,]?\d*)\s*(kg|g|gr|gramas)(?!\s*gratis)\b'
df[['quantidade', 'unidade']] = df['nome'].apply(lambda text: mask_qnt(text, pattern)).apply(pd.Series)
df['qnt_gramas'] = df[['quantidade', 'unidade']].apply(convert_to_gr, axis=1)

pattern = r'(\d+)\s*(caps|cap|cáps|v-caps|cápsulas|capsules|cáp|comprimidos|comps|comp|capsulas|barras|soft|softgel|sachês|doses|dose|tabs|tabletes|tablets|gotas)\b'
df[['quantidade_formato', 'formato']] = df['nome'].apply(lambda text: mask_qnt(text, pattern)).apply(pd.Series)
df['formato'] = df['formato'].apply(substituir_por_comprimidos)

pattern = r'(\d+)\s*(caixa|display\s*c/|unidades|unid|un|saches|caixas|x)\b'
df[['quantidade_formato_2', 'formato_2']] = df['nome'].apply(lambda text: mask_qnt(text, pattern)).apply(pd.Series)

In [14]:
def calcular_divisao_e_lidar_com_negativos(row):
    resultado = row['preco_numeric'] / row['qnt_gramas']
    if resultado < 0:
        return np.nan  # Substitui valores negativos por NaN
    return resultado

# Aplicar a função para calcular a divisão e lidar com valores negativos
df['resultado_divisao'] = df.apply(calcular_divisao_e_lidar_com_negativos, axis=1)
df['resultado_divisao'] 

0         0.072167
1         0.105456
2         0.077667
3              NaN
4         0.144333
5         0.055444
6         0.199900
7         0.199900
8         0.199000
9         0.333300
10        0.259800
11        0.396133
12        0.399333
13             NaN
14        0.299667
15        0.233000
16        0.332667
17             NaN
18             NaN
19        0.232667
20        0.023300
21        0.029900
22        0.399667
23        0.199989
24        0.194433
25        0.133322
26        0.099989
27        0.083322
28             NaN
29        0.555500
30        0.555500
31        0.555500
32        0.366633
33        0.259980
34        0.366600
35        0.466600
36        0.555444
37             NaN
38             NaN
39        0.199967
40        0.266600
41        0.291583
42             NaN
43             NaN
44             NaN
45             NaN
46             NaN
47        1.055500
48        1.055500
49        1.055500
50        1.438833
51        1.666611
52          

In [23]:
filtro = df[['quantidade', 'unidade', 'quantidade_formato', 'formato', 'quantidade_formato_2', 'formato_2']].isnull().all(axis=1)
df_filter = df[filtro]
df_filter[['nome', 'quantidade', 'unidade', 'quantidade_formato', 'formato', 'quantidade_formato_2', 'formato_2']]

,nome,quantidade,unidade,quantidade_formato,formato,quantidade_formato_2,formato_2
17,l-carnitine 2000,None,None,None,None,None,None
60,combo kimera thermo + kimera night,None,None,None,None,None,None
63,combo kimera ultimate + kiron + kimera night,None,None,None,None,None,None
67,combo emagrecimento ultimate,None,None,None,None,None,None
89,combo kimera woman + kimera night + kimera sma...,None,None,None,None,None,None
91,combo kimera thermo + kimera night + kimera sm...,None,None,None,None,None,None
94,combo snacks proteicos,None,None,None,None,None,None
95,best vegan protein tiramissú,None,None,None,None,None,None
96,best vegan protein muffin com morango & banana,None,None,None,None,None,None
97,best vegan protein bolo de banana,None,None,None,None,None,None


In [15]:
# df[['nome', 'quantidade', 'unidade', 'quantidade_formato', 'formato', 'quantidade_formato_2', 'formato_2']]

In [6]:
df[['quantidade', 'unidade']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053 entries, 0 to 1052
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   quantidade  487 non-null    object
 1   unidade     487 non-null    object
dtypes: object(2)
memory usage: 16.6+ KB


In [17]:
# df[['titulo', 'quantidade', 'unidade']]

In [18]:
# df[[, 'quantidade', 'unidade']]